In [11]:
import pandas as pd

In [12]:
#data_dis = pd.read_csv("distemist_subtrack1_training_mentions.tsv", delimiter="\t")
data_dis = pd.read_csv("distemist_subtrack1_test_mentions.tsv", delimiter="\t")
data_dis

,filename,mark,label,off0,off1,span
0,es-S1138-123X2005000200006-2,T1,ENFERMEDAD,13,27,labio leporino
1,es-S1138-123X2005000200006-2,T2,ENFERMEDAD,30,55,fisura palatina bilateral
2,es-S1138-123X2005000200006-2,T3,ENFERMEDAD,60,95,premaxila estaba protruida y rotada
3,es-S0211-69952011000100019-1,T1,ENFERMEDAD,644,659,derrame pleural
4,es-S0211-69952011000100019-1,T2,ENFERMEDAD,45,64,glucogenosis tipo V
...,...,...,...,...,...,...
2593,es-S0212-16112011000300031-1,T7,ENFERMEDAD,845,868,pinza aorto-mesentérica
2594,es-S0212-16112011000300031-1,T8,ENFERMEDAD,1034,1074,trastorno del comportamiento alimentario
2595,es-S0212-16112011000300031-1,T9,ENFERMEDAD,2321,2339,hipotonía gástrica
2596,es-S0212-16112011000300031-1,T10,ENFERMEDAD,528,547,patología digestiva


In [13]:
f_out = open("DisTEMIST_testing.json", 'w', encoding="UTF-8")
f_out.write('{"version": "0.1.0",\n"data": [')
test_path = "text_files_testing"
test_files = os.listdir(test_path)
for f in test_files:
    with open(test_path+'/'+f, "r", encoding="UTF-8") as ftest:
        texto = ftest.read()
    fl = data_dis[data_dis['filename'] == f[:-4]].sort_values('off0', ascending=True)
    diseases = []
    for i in range(len(fl)):
        diseases.append(fl['span'].iloc[i])
    for p in diseases:        
        tagged = [a+"___I#-#D#I#S#E#A#S#E" for a in p.split()]
        tagged[0] = tagged[0].replace('I#-#D#I#S#E#A#S#E', 'B#-#D#I#S#E#A#S#E')
        txt_tag = ' '.join( tagged )
        #texto = re.sub(texto[start:end],txt_tag,texto)
        texto = texto.replace(p, txt_tag)
        texto = texto.replace('. ',' . ')
        texto = texto.replace('.\n',' . ')
        texto = texto.replace(', ',' , ')
        texto = texto.replace(': ',' : ')
        texto = texto.replace('! ',' ! ')
        texto = texto.replace('? ',' ? ')
        texto = texto.replace('; ',' ; ')
        texto = texto.replace(') ',' ) ')
        texto = texto.replace('" ',' \" ')
        texto = texto.replace(' (',' ( ')
        texto = texto.replace(' "',' \" ')
        texto = texto.replace('  ', ' ')
    texto = texto.split()
    ner_tags = []
    tokens = []
    for w in texto:
        #if w.endswith('B-DISEASE') or w.endswith('I-DISEASE'):
            #wp = w.split('___')
            #tokens.append(wp[0])
            #ner_tags.append(wp[1])
        
        
        '''
        #validate double tags
        if "___B#-#D#I#S#E#A#S#E___B#-#D#I#S#E#A#S#E" in w:
            #print(w)
            w = w.replace("___B#-#D#I#S#E#A#S#E___B#-#D#I#S#E#A#S#E","___B#-#D#I#S#E#A#S#E")
        elif "___B#-#D#I#S#E#A#S#E___I#-#D#I#S#E#A#S#E" in w:
            #print(w)
            w = w.replace("___B#-#D#I#S#E#A#S#E___I#-#D#I#S#E#A#S#E","___I#-#D#I#S#E#A#S#E")
        elif "___I#-#D#I#S#E#A#S#E___B#-#D#I#S#E#A#S#E" in w:
            #print(w)
            w = w.replace("___I#-#D#I#S#E#A#S#E___B#-#D#I#S#E#A#S#E","___B#-#D#I#S#E#A#S#E")
        elif "___I#-#D#I#S#E#A#S#E___I#-#D#I#S#E#A#S#E" in w:
            #print(w)
            w = w.replace("___I#-#D#I#S#E#A#S#E___I#-#D#I#S#E#A#S#E","___I#-#D#I#S#E#A#S#E")
        '''
        
        if "B#-#D#I#S#E#A#S#E" in w:
            wp = w.replace("___B#-#D#I#S#E#A#S#E", "")
            wp = wp.replace("___I#-#D#I#S#E#A#S#E", "") #To clean multiple tags
            tokens.append(wp)
            ner_tags.append("B-DISEASE")
        elif "I#-#D#I#S#E#A#S#E" in w:
            wp = w.replace("___I#-#D#I#S#E#A#S#E", "")
            tokens.append(wp)
            ner_tags.append("I-DISEASE")
        else:
            tokens.append(w)
            ner_tags.append("O")
        if w == ".":
            dicc = {"ner_tags": ner_tags, "tokens": tokens}
            f_out.write(str(dicc)+',\n')
            ner_tags = []
            tokens = []
    #print(tokens, len(tokens))
    #print(ner_tags, len(ner_tags))
f_out.write("]}")
f_out.close()

<IPython.core.display.Javascript object>

## Procesando el diccionario de enfermedades

In [10]:
dic = open("Diccionario_Diseases.txt", "r", encoding="ANSI")
f = open("Diccionario_BIO.json", "w", encoding="ANSI")
for dis in dic:
    tokens = dis.split()
    ner_tags = ["I-DISEASE" for t in tokens]
    ner_tags[0] = "B-DISEASE"
    f.write('{"ner_tags":' + str(ner_tags) + ', "tokens:"' + str(tokens) + '},\n' )
f.close()
dic.close()